In [1]:
import pandas as pd
from pathlib import Path
import numpy as np

In [3]:
# Load data
file_path = Path("Resources/hmda_2017_nationwide_all-records_codes.csv")
df = pd.read_csv(file_path)

In [4]:
df.count()

as_of_year                        14285496
respondent_id                     14285496
agency_code                       14285496
loan_type                         14285496
property_type                     14285496
loan_purpose                      14285496
owner_occupancy                   14285496
loan_amount_000s                  14280460
preapproval                       14285496
action_taken                      14285496
msamd                             12494446
state_code                        14088338
county_code                       14039899
census_tract_number               14001194
applicant_ethnicity               14285496
co_applicant_ethnicity            14285496
applicant_race_1                  14285496
applicant_race_2                     90382
applicant_race_3                      6990
applicant_race_4                      1644
applicant_race_5                      1084
co_applicant_race_1               14285496
co_applicant_race_2                  30414
co_applican

In [5]:
# Drop unnecessary columns
df = df.drop(columns=["as_of_year","applicant_race_2", "applicant_race_3", "applicant_race_4", "applicant_race_5",
                        "co_applicant_race_2", "co_applicant_race_3", "co_applicant_race_4", "co_applicant_race_5",
                        "denial_reason_2", "denial_reason_3", "rate_spread", "edit_status", "sequence_number", 
                        "application_date_indicator"])
df.count()

respondent_id                     14285496
agency_code                       14285496
loan_type                         14285496
property_type                     14285496
loan_purpose                      14285496
owner_occupancy                   14285496
loan_amount_000s                  14280460
preapproval                       14285496
action_taken                      14285496
msamd                             12494446
state_code                        14088338
county_code                       14039899
census_tract_number               14001194
applicant_ethnicity               14285496
co_applicant_ethnicity            14285496
applicant_race_1                  14285496
co_applicant_race_1               14285496
applicant_sex                     14285496
co_applicant_sex                  14285496
applicant_income_000s             12389632
purchaser_type                    14285496
denial_reason_1                    1411970
hoepa_status                      14285496
lien_status

In [6]:
# respondent_id is identifier - varchar

In [7]:
# agency_code - change to string/object - varchar
df['agency_code'] = df['agency_code'].replace([1,2,3,5,7,9],['OCC','FRS','FDIC','NCUA','HUD','CFPB'])
df["agency_code"].value_counts()

HUD     7083670
CFPB    4222898
FDIC    1128554
NCUA    1020531
OCC      463411
FRS      366432
Name: agency_code, dtype: int64

In [8]:
# loan_type - change to string/object - varchar
df['loan_type'] = df['loan_type'].replace([1,2,3,4],['Conventional','FHA-Insured','VA-Guaranteed','FSA/RHS'])
df["loan_type"].value_counts()

Conventional     9950346
FHA-Insured      2659118
VA-Guaranteed    1402700
FSA/RHS           273332
Name: loan_type, dtype: int64

In [9]:
# property_type - change to string/object - varchar
df['property_type'] = df['property_type'].replace([1,2,3],['One to Four-Family','Manufactured','Multifamily'])
df["property_type"].value_counts()

One to Four-Family    13706551
Manufactured            527095
Multifamily              51850
Name: property_type, dtype: int64

In [10]:
# loan_purpose - change to string/object - varchar
df['loan_purpose'] = df['loan_purpose'].replace([1,2,3],['Home Purchase','Home Improvement','Refinancing'])
df["loan_purpose"].value_counts()

Home Purchase       7703630
Refinancing         5490094
Home Improvement    1091772
Name: loan_purpose, dtype: int64

In [11]:
# owner_occupancy - change to string/object - varchar
df['owner_occupancy'] = df['owner_occupancy'].replace([1,2,3],['Owner Occupied','Not Owner Occupied','Not Applicable'])
df["owner_occupancy"].value_counts()

Owner Occupied        12887163
Not Owner Occupied     1335445
Not Applicable           62888
Name: owner_occupancy, dtype: int64

In [12]:
# loan_amount_000s - some null values here - odd - keep the same - integer

In [13]:
# preapproval - change to string/object - varchar
df['preapproval'] = df['preapproval'].replace([1,2,3],['Preapproval Requested','Preapproval Not Requested','Not Applicable'])
df["preapproval"].value_counts()

Not Applicable               11859707
Preapproval Not Requested     1944405
Preapproval Requested          481384
Name: preapproval, dtype: int64

In [14]:
# action_taken - change to string/object - varchar
df['action_taken'] = df['action_taken'].replace([1,2,3,4,5,6,7,8],['Loan Originated','Application Approved but not Accepted','Application Denied by Financial Institution','Application Withdrawn by Applicant','File Closed for Incompleteness','Loan Purchased by the Institution','Preapproval Request Denied by Financial Institution','Preapproval Request Approved but not Accepted'])
df["action_taken"].value_counts()

Loan Originated                                        7339057
Loan Purchased by the Institution                      2086676
Application Denied by Financial Institution            2009743
Application Withdrawn by Applicant                     1696289
File Closed for Incompleteness                          601148
Application Approved but not Accepted                   409797
Preapproval Request Denied by Financial Institution     106680
Preapproval Request Approved but not Accepted            36106
Name: action_taken, dtype: int64

In [15]:
# create new column from action taken with numbers to represent approved, denied, and other
# action_taken_summary - integer I think
action = {
    'Loan Originated': 1, 
    'Loan Purchased by the Institution': 1, 
    'Application Denied by Financial Institution': 0,
    'Application Withdrawn by Applicant': 2,
    'File Closed for Incompleteness': 2,
    'Application Approved but not Accepted': 0,
    'Preapproval Request Denied by Financial Institution': 0,
    'Preapproval Request Approved but not Accepted': 0}

df["action_taken_summary"] = df["action_taken"].map(action)

In [16]:
# value counts of action_taken_summary
df["action_taken_summary"].value_counts()

1    9425733
0    2562326
2    2297437
Name: action_taken_summary, dtype: int64

In [17]:
# msamd is area - change all null values to 0??? float/decimal
df['msamd'] = df['msamd'].fillna(0)

In [18]:
# create regions using state_code - Two letter code - varchar(2)
states_to_regions = {
    53: 'NW', 
    41: 'NW', 
    6: 'W',
    32: 'W',
    16: 'NW',
    30: 'NW',
    56: 'NW',
    49: 'SW',
    8: 'SW',
    2: 'NW',
    15: 'W',
    23: 'NE',
    50: 'NE',
    36: 'MA',
    33: 'NE',
    25: 'NE',
    44: 'NE',
    9: 'NE',
    34: 'MA',
    42: 'MA', 
    38: 'MW',
    46: 'MW',
    31: 'MW',
    20: 'MW',
    27: 'MW',
    19: 'MW',
    29: 'MW',
    55: 'MW',
    17: 'MW',
    26: 'MW',
    18: 'MW',
    39: 'MW',
    54: 'MA',
    11: 'MA',
    24: 'MA',
    51: 'MA',
    21: 'MW',
    47: 'SE',
    37: 'SE',
    28: 'SE',
    5: 'SE',
    22: 'SE',
    1: 'SE',
    13: 'SE',
    45: 'SE',
    12: 'SE',
    10: 'MA',
    4: 'SW',
    35: 'SW',
    40: 'SW',
    48: 'SW',
    72: 'SE',
    78: 'SE'}

df["Region"] = df["state_code"].map(states_to_regions)
df["Region"].value_counts()

SE    3204049
MW    3006716
SW    2410820
MA    2120637
W     1937914
NW     806678
NE     601524
Name: Region, dtype: int64

In [19]:
# state_code - change to string/object - varchar(2) - includes Puerto Rico and Virgin Islands
df['state_code'] = df['state_code'].replace([1,2,5,4,6,8,9,11,10,12,13,15,19,16,17,18,20,21,22,25,24,23,26,27,29,28,30,37,38,31,33,34,35,32,36,39,40,41,42,72,44,45,46,47,48,49,51,50,53,55,54,56,78],
    ['AL','AK','AR','AZ','CA','CO','CT','DC','DE','FL','GA','HI','IA','ID','IL','IN','KS','KY','LA','MA','MD','ME','MI','MN','MO','MS','MT','NC','ND','NE','NH','NJ','NM','NV','NY','OH','OK','OR','PA','PR','RI','SC','SD','TN','TX','UT','VA','VT','WA','WI','WV','WY','VI'])
df["state_code"].value_counts()

CA    1714459
TX    1148206
FL    1018763
IL     502511
GA     501310
PA     473757
NC     464109
OH     448269
NY     446902
MI     437181
AZ     428411
VA     411507
CO     404517
WA     402196
NJ     349563
TN     326416
MD     301879
IN     292152
MA     282546
MO     277843
MN     269551
SC     242772
WI     237542
OR     211344
AL     209002
UT     198425
NV     178587
LA     173079
KY     173004
OK     150838
CT     129401
IA     127446
AR     125392
KS     107563
MS     101384
ID      92755
NM      80423
NE      74966
NH      65016
ME      58188
WV      56407
DE      49695
MT      45597
HI      44868
RI      44038
PR      41775
SD      33167
DC      30927
AK      28632
WY      26154
ND      25521
VT      22335
VI         47
Name: state_code, dtype: int64

In [20]:
# county_code - 3 digit code -leave as is integer

In [21]:
# census_tract_number - code - leave as float/decimal

In [22]:
# applicant_ethnicity - change to string/object - varchar
df['applicant_ethnicity'] = df['applicant_ethnicity'].replace([1,2,3,4,5],['Hispanic or Latino','Not Hispanic or Latino','Info not Provided by Applicant','Not Applicable','No Co-Applicant'])
df["applicant_ethnicity"].value_counts()

Not Hispanic or Latino            9877958
Info not Provided by Applicant    1543770
Not Applicable                    1520462
Hispanic or Latino                1343306
Name: applicant_ethnicity, dtype: int64

In [23]:
# co_applicant_ethnicity - change to string/object - varchar
df['co_applicant_ethnicity'] = df['co_applicant_ethnicity'].replace([1,2,3,4,5],['Hispanic or Latino','Not Hispanic or Latino','Info not Provided by Applicant','Not Applicable','No Co-Applicant'])
df["co_applicant_ethnicity"].value_counts()

No Co-Applicant                   7465120
Not Hispanic or Latino            4282121
Not Applicable                    1264895
Info not Provided by Applicant     738020
Hispanic or Latino                 535340
Name: co_applicant_ethnicity, dtype: int64

In [24]:
# applicant_race_1 - change to string/object - varchar
df['applicant_race_1'] = df['applicant_race_1'].replace([1,2,3,4,5,6,7,8],['American Indian or Alaska Native','Asian','Black or African American','Native Hawaiian or Other Pacific Islander','White','Information not Provided','Not Applicable','No Co-Applicant'])
df["applicant_race_1"].value_counts()

White                                        9267426
Information not Provided                     1641022
Not Applicable                               1513818
Black or African American                    1002556
Asian                                         692059
American Indian or Alaska Native              109588
Native Hawaiian or Other Pacific Islander      59027
Name: applicant_race_1, dtype: int64

In [25]:
# co_applicant_race_1 - change to string/object - varchar
df['co_applicant_race_1'] = df['co_applicant_race_1'].replace([1,2,3,4,5,6,7,8],['American Indian or Alaska Native','Asian','Black or African American','Native Hawaiian or Other Pacific Islander','White','Information not Provided','Not Applicable','No Co-Applicant'])
df["co_applicant_race_1"].value_counts()

No Co-Applicant                              7465119
White                                        4153379
Not Applicable                               1261891
Information not Provided                      768723
Asian                                         314750
Black or African American                     247658
American Indian or Alaska Native               37661
Native Hawaiian or Other Pacific Islander      36315
Name: co_applicant_race_1, dtype: int64

In [26]:
# applicant_sex - change to string/object - varchar
df['applicant_sex'] = df['applicant_sex'].replace([1,2,3,4,5],['Male','Female','Information not Provided','Not Applicable','No Co-Applicant'])
df["applicant_sex"].value_counts()

Male                        8047593
Female                      3760924
Not Applicable              1512585
Information not Provided     964394
Name: applicant_sex, dtype: int64

In [27]:
# co_applicant_sex - change to string/object - varchar
df['co_applicant_sex'] = df['co_applicant_sex'].replace([1,2,3,4,5],['Male','Female','Information not Provided','Not Applicable','No Co-Applicant'])
df["co_applicant_sex"].value_counts()

No Co-Applicant             7465121
Female                      3946984
Not Applicable              1261521
Male                        1145317
Information not Provided     466553
Name: co_applicant_sex, dtype: int64

In [28]:
# applicant_income_000s - fill na values with 0? - leave as integer
df['applicant_income_000s'] = df['applicant_income_000s'].fillna(0)

In [29]:
# purchaser_type - change to string/object - varchar
df['purchaser_type'] = df['purchaser_type'].replace([0,1,2,3,4,5,6,7,8,9],['Loan Not Originated','FNMA','GNMA','FHLMC','FAMC','Private Securitization','Commercial Bank, Savings Bank, or Savings Assoc','Life Insurance Company, Credit Union, Mortgage Bank, or Finance Company','Affiliate Institution','Other Type of Purchaser'])
df["purchaser_type"].value_counts()

Loan Not Originated                                                        7279078
FNMA                                                                       1842055
GNMA                                                                       1730827
FHLMC                                                                      1290442
Life Insurance Company, Credit Union, Mortgage Bank, or Finance Company     834022
Commercial Bank, Savings Bank, or Savings Assoc                             756799
Other Type of Purchaser                                                     407372
Affiliate Institution                                                        90030
Private Securitization                                                       53534
FAMC                                                                          1337
Name: purchaser_type, dtype: int64

In [30]:
# denial_reason_1 - fill na values with 0?
df['denial_reason_1'] = df['denial_reason_1'].fillna(0)

In [31]:
df["denial_reason_1"].value_counts()

0.0    12873526
3.0      392096
1.0      384777
4.0      220014
7.0      179330
9.0      126294
6.0       49326
5.0       37848
2.0       21008
8.0        1277
Name: denial_reason_1, dtype: int64

In [32]:
# denial_reason_1 - change the names to reflect denial reasons - varchar
df['denial_reason_1'] = df['denial_reason_1'].replace([0,1,2,3,4,5,6,7,8,9],['Approved - N/A','Debt-to-Income Ratio','Employment History','Credit History','Collateral','Insufficient Cash (Downpayment, Closing Costs)','Unverifiable Information','Credit Application Incomplete','Mortgage Insurance Denied','Other'])
df["denial_reason_1"].value_counts()

Approved - N/A                                    12873526
Credit History                                      392096
Debt-to-Income Ratio                                384777
Collateral                                          220014
Credit Application Incomplete                       179330
Other                                               126294
Unverifiable Information                             49326
Insufficient Cash (Downpayment, Closing Costs)       37848
Employment History                                   21008
Mortgage Insurance Denied                             1277
Name: denial_reason_1, dtype: int64

In [33]:
# hoepa_status - change to string/object - varchar
df['hoepa_status'] = df['hoepa_status'].replace([1,2],['HOEPA Loan','Not a HOEPA Loan'])
df["hoepa_status"].value_counts()

Not a HOEPA Loan    14281893
HOEPA Loan              3603
Name: hoepa_status, dtype: int64

In [34]:
# lien_status - change to string/object - varchar
df['lien_status'] = df['lien_status'].replace([1,2,3,4],['Secured by First Lien','Secured by a Subordinate Loan','Not Secured by Lien','Not Applicable (Purchased Loan)'])
df["lien_status"].value_counts()

Secured by First Lien              11389560
Not Applicable (Purchased Loan)     2086676
Secured by a Subordinate Loan        407390
Not Secured by Lien                  401870
Name: lien_status, dtype: int64

In [35]:
# population - in tract - leave as integer
df["population"].value_counts()

4853.0    5992
6083.0    5771
5114.0    5649
4428.0    5595
5019.0    5515
          ... 
59.0         1
510.0        1
724.0        1
22.0         1
87.0         1
Name: population, Length: 9551, dtype: int64

In [36]:
# minority_population - percentage - leave as integer

In [37]:
# hud_median_family_income - in $ - leave as integer

In [38]:
# tract_to_msamd_income - percentage - leave as integer

In [39]:
# number_of_owner_occupied_units - leave as integer
df["number_of_owner_occupied_units"].value_counts()

1265.0    12682
1149.0    12651
1165.0    12579
1281.0    12562
1440.0    12533
          ...  
19.0         72
27.0         69
21.0         54
3.0          18
2.0           9
Name: number_of_owner_occupied_units, Length: 3356, dtype: int64

In [ ]:
# number_of_1_to_4_family_units - leave as integer

In [40]:
# check the null values in the columns now
df.isnull().sum()

respondent_id                          0
agency_code                            0
loan_type                              0
property_type                          0
loan_purpose                           0
owner_occupancy                        0
loan_amount_000s                    5036
preapproval                            0
action_taken                           0
msamd                                  0
state_code                        197158
county_code                       245597
census_tract_number               284302
applicant_ethnicity                    0
co_applicant_ethnicity                 0
applicant_race_1                       0
co_applicant_race_1                    0
applicant_sex                          0
co_applicant_sex                       0
applicant_income_000s                  0
purchaser_type                         0
denial_reason_1                        0
hoepa_status                           0
lien_status                            0
population      

In [41]:
# drop all na values
df = df.dropna()

In [42]:
df.count()

respondent_id                     13997124
agency_code                       13997124
loan_type                         13997124
property_type                     13997124
loan_purpose                      13997124
owner_occupancy                   13997124
loan_amount_000s                  13997124
preapproval                       13997124
action_taken                      13997124
msamd                             13997124
state_code                        13997124
county_code                       13997124
census_tract_number               13997124
applicant_ethnicity               13997124
co_applicant_ethnicity            13997124
applicant_race_1                  13997124
co_applicant_race_1               13997124
applicant_sex                     13997124
co_applicant_sex                  13997124
applicant_income_000s             13997124
purchaser_type                    13997124
denial_reason_1                   13997124
hoepa_status                      13997124
lien_status

In [43]:
# This is as clean as I can get the raw data without affecting it negatively

In [44]:
df.dtypes

respondent_id                      object
agency_code                        object
loan_type                          object
property_type                      object
loan_purpose                       object
owner_occupancy                    object
loan_amount_000s                  float64
preapproval                        object
action_taken                       object
msamd                             float64
state_code                         object
county_code                       float64
census_tract_number               float64
applicant_ethnicity                object
co_applicant_ethnicity             object
applicant_race_1                   object
co_applicant_race_1                object
applicant_sex                      object
co_applicant_sex                   object
applicant_income_000s             float64
purchaser_type                     object
denial_reason_1                    object
hoepa_status                       object
lien_status                       

In [45]:
save_path = "Resources/cleaned_raws.csv"
df.to_csv(save_path, index=False)